## 4. K-mean clustering

### 4.1. Load libraries

In [28]:
import pandas as pd
import numpy as np
import ipywidgets as widgets

In [29]:
import seaborn as sns

from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, FactorRange
from bokeh.io import show, output_file
from bokeh.models.tickers import FixedTicker
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
from bokeh.tile_providers import CARTODBPOSITRON

In [30]:
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy import stats

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA

### 4.2. Functions

In [32]:
def kcolor(num):
    global kolors
    color_code = ['red','green','blue','orange','purple','brown','cadetblue','maroon','olive']
    kolors = len(color_code)
    if num in np.arange(0, len(color_code)):
        pickolor = color_code[num]
    else:
        pickolor = 'black'
    
    return pickolor
        

### 4.3. Reading Data

 This widget will set the number of clusters.

In [33]:
# The limit is the number of the colors defined in kcolor function above (clusters will be color labeled)

kcolor(1)

wg_cluster = widgets.IntSlider(
    max = kolors,
    min = 1,
    value = 5
)

display(wg_cluster)

A Jupyter Widget

In [34]:
pc123_df = pd.read_csv('data/plots.csv', index_col=0)
vec123_df = pd.read_csv('data/vectors.csv', index_col=0)
feat_dict = pd.read_csv('data/Col_Dict.csv', index_col=0)

In [35]:
vec123_full = vec123_df.merge(feat_dict, left_index=True, right_on='brief_header', how='left')
vec123_full.index = vec123_full['brief_header']
vec123_full.drop(['brief_header','Ref'], axis=1, inplace=True)
#vec123_full

In [36]:
pc12_df = pc123_df.drop(['PC3'], axis=1)
pc13_df = pc123_df.drop(['PC2'], axis=1)
vec12_df = vec123_df.drop(['PC3'], axis=1)
vec13_df = vec123_df.drop(['PC2'], axis=1)

### 4.4. K-mean clustering

In [37]:

pc12_a = np.array(pc12_df)

# Create a KMeans instance: model
model = KMeans(n_clusters=wg_cluster.value)

# Fit model to points
model.fit(pc12_a)

# Determine the cluster labels of new_points: labels
labels = model.predict(pc12_a)

color_list = []
for i in np.arange(len(labels)):
    color_list.append(kcolor(labels[i]))

pc12_df['cluster'] = color_list

In [38]:

pc13_a = np.array(pc13_df)

# Create a KMeans instance: model
model = KMeans(n_clusters=wg_cluster.value)

# Fit model to points
model.fit(pc13_a)

# Determine the cluster labels of new_points: labels
labels = model.predict(pc13_a)

color_list = []
for i in np.arange(len(labels)):
    color_list.append(kcolor(labels[i]))

pc13_df['cluster'] = color_list

In [39]:
vec12_a = np.array(vec12_df)

# Create a KMeans instance: model
model = KMeans(n_clusters=wg_cluster.value)

# Fit model to points
model.fit(vec12_a)

# Determine the cluster labels of new_points: labels
labels = model.predict(vec12_a)

color_list = []
for i in np.arange(len(labels)):
    color_list.append(kcolor(labels[i]))

vec12_df['cluster'] = color_list

In [40]:
vec13_a = np.array(vec13_df)

# Create a KMeans instance: model
model = KMeans(n_clusters=wg_cluster.value)

# Fit model to points
model.fit(vec13_a)

# Determine the cluster labels of new_points: labels
labels = model.predict(vec13_a)

color_list = []
for i in np.arange(len(labels)):
    color_list.append(kcolor(labels[i]))

vec13_df['cluster'] = color_list

In [41]:
vec12_df = vec12_df.merge(feat_dict, left_index=True, right_on='brief_header', how='left')
vec12_df.index = vec12_df['brief_header']
vec12_df.drop(['brief_header','Ref'], axis=1, inplace=True)
# vec12_df

In [42]:
vec13_df = vec13_df.merge(feat_dict, left_index=True, right_on='brief_header', how='left')
vec13_df.index = vec13_df['brief_header']
vec13_df.drop(['brief_header','Ref'], axis=1, inplace=True)
# vec13_df

### 4.5. Plotting results
Results will appear in new tab/window

In [115]:
output_file("pc1vs2m.html")


# SourceP and SourceV are hover texts for data points and eigenvectors respectively
sourcep = ColumnDataSource(data=dict(
    x=pc12_df['PC1'],
    y=pc12_df['PC2'],
    desc=list(pc12_df.index),
))

sourcev = ColumnDataSource(data=dict(
    x=vec123_df['PC1'],
    y=vec123_df['PC2'],
    desc=vec123_full['description'],
))


hover = HoverTool(tooltips=[("", "@desc"),])

p = figure(title="PC1 vs PC2 - Not Clustered",
           plot_width=1300, plot_height=700,
           x_axis_label='Principal Component 1',
          y_axis_label='Principal Component 2', 
           tools=[hover])

pc_x = pc12_df['PC1']
pc_y = pc12_df['PC2']
pc_c = pc12_df['cluster']

p.circle(pc_x, pc_y, color=None, fill_color='red', size=9, fill_alpha = 0.5, source = sourcep)

rescale = 50
pc1_vx = rescale * vec123_df['PC1']
pc1_vy2 = rescale * vec123_df['PC2']
pc_cv = vec12_df['cluster']


p.segment(0,0, pc1_vx, pc1_vy2, line_color="black", line_width = 1, line_alpha = 0.1)

p.triangle(pc1_vx, pc1_vy2, color = None, size=5, fill_color='black', fill_alpha = 0.5, angle=-np.abs(pc1_vx) * np.arctan(pc1_vx/pc1_vy2), source = sourcev)

show(p)

/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND

In [113]:
output_file("pc1vs2k.html")


# SourceP and SourceV are hover texts for data points and eigenvectors respectively
sourcep = ColumnDataSource(data=dict(
    x=pc12_df['PC1'],
    y=pc12_df['PC2'],
    desc=list(pc12_df.index),
))

sourcev = ColumnDataSource(data=dict(
    x=vec123_df['PC1'],
    y=vec123_df['PC2'],
    desc=vec123_full['description'],
))


hover = HoverTool(tooltips=[("", "@desc"),])

p = figure(title="PC1 vs PC2 - K-mean clustered",
           plot_width=1300, plot_height=700,
           x_axis_label='Principal Component 1',
          y_axis_label='Principal Component 2', 
           tools=[hover])

pc_x = pc12_df['PC1']
pc_y = pc12_df['PC2']
pc_c = pc12_df['cluster']

p.circle(pc_x, pc_y, color=None, fill_color=pc_c, size=9, fill_alpha = 0.5, source = sourcep)

rescale = 50
pc1_vx = rescale * vec123_df['PC1']
pc1_vy2 = rescale * vec123_df['PC2']
pc_cv = vec12_df['cluster']


p.segment(0,0, pc1_vx, pc1_vy2, line_color="black", line_width = 1, line_alpha = 0.1)

p.triangle(pc1_vx, pc1_vy2, color = None, size=5, fill_color=pc_cv, fill_alpha = 0.5, angle=-np.abs(pc1_vx) * np.arctan(pc1_vx/pc1_vy2), source = sourcev)

show(p)

/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND

In [114]:
output_file("pc1vs3k.html")


# SourceP and SourceV are hover texts for data points and eigenvectors respectively
sourcep = ColumnDataSource(data=dict(
    x=pc13_df['PC1'],
    y=pc13_df['PC3'],
    desc=list(pc12_df.index),
))

sourcev = ColumnDataSource(data=dict(
    x=vec123_df['PC1'],
    y=vec123_df['PC3'],
    desc=vec123_full['description'],
))


hover = HoverTool(tooltips=[("", "@desc"),])

p = figure(title="PC1 vs PC3 - K-mean clustered",
           plot_width=1300, plot_height=700,
           x_axis_label='Principal Component 1',
          y_axis_label='Principal Component 3', 
           tools=[hover])

pc_x = pc13_df['PC1']
pc_y = pc13_df['PC3']
pc_c = pc13_df['cluster']

p.circle(pc_x, pc_y, color=None, fill_color=pc_c, size=9, fill_alpha = 0.5, source = sourcep)

rescale = 50
pc1_vx = rescale * vec123_df['PC1']
pc1_vy2 = rescale * vec123_df['PC3']
pc_cv = vec12_df['cluster']


p.segment(0,0, pc1_vx, pc1_vy2, line_color="black", line_width = 1, line_alpha = 0.1)

p.triangle(pc1_vx, pc1_vy2, color = None, size=5, fill_color=pc_cv, fill_alpha = 0.5, angle=-np.abs(pc1_vx) * np.arctan(pc1_vx/pc1_vy2), source = sourcev)

show(p)

/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/ghfmhf/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND

In [109]:
import folium
cty_map = pd.read_csv('data/indus_dist.csv', index_col=0)
map_df = cty_map.merge(pc12_df, left_on='county', right_index=True, how='left')
map_df.drop(['Avg_Dist','PC1','PC2'], axis=1, inplace=True)


map_2 = folium.Map(location=[36.5, -86],
                   tiles='Stamen Toner',
                   zoom_start=7)
for i in np.arange(map_df.shape[0]):
    folium.CircleMarker([map_df.iloc[i,1], map_df.iloc[i,2]],
                    radius=7,
                    color=map_df.iloc[i,3],
                    fill_color=map_df.iloc[i,3],
                   ).add_to(map_2)
map_2


In [90]:
clus = widgets.Dropdown(
    options=list(vec12_df.iloc[:,2].unique()),
    description='Feature:',
    disabled=False,
)
display(clus)

A Jupyter Widget

In [118]:
pd.DataFrame(vec12_df[vec12_df['cluster']==clus.value]['description'].sort_values())

,description
brief_header,
cp_id_fm_02,Cropland idle or used for cover crops or soil ...
cp_id_fm_07,Cropland idle or used for cover crops or soil ...
cp_id_fm_12,Cropland idle or used for cover crops or soil ...
cp_gz_acr_07x,Cropland used only for pasture or grazing acre...
cp_gz_acr_12,Cropland used only for pasture or grazing acre...
fm_val_049_12,Farms by value group 1 to 49999 -2012
fm_val_100_12,Farms by value group 100000 to 199999 -2012
fm_val_050_12,Farms by value group 50000 to 99999 - 2012
lnd_acr_07,Land in farms (acres) - 2007


In [92]:
cty = widgets.Dropdown(
    options=list(pc12_df.iloc[:,2].unique()),
    description='County:',
    disabled=False,
)
display(cty)

A Jupyter Widget

In [107]:
pd.DataFrame(pc12_df[pc12_df['cluster']==cty.value].index.sort_values())

,0
0,blount
1,dickson
2,jefferson
3,montgomery
4,putnam
5,robertson
6,rutherford
7,sullivan
8,sumner
9,wilson


In [66]:
cty_map = pd.read_csv('data/indus_dist.csv', index_col=0)


cty_map.head(3)


,county,latitude,longitude,Avg_Dist
16494,anderson,36.20,-84.04,32775.820230
16173,bedford,35.59,-86.35,4503.073082
16773,benton,36.23,-88.08,6487.141318


In [67]:
pc12_df.head(3)

,PC1,PC2,cluster
anderson,-7.332806,2.383207,green
bedford,16.606917,-1.754641,blue
benton,-4.443341,-2.766427,green


In [70]:
map_df = cty_map.merge(pc12_df, left_on='county', right_index=True, how='left')
map_df.drop(['Avg_Dist','PC1','PC2'], axis=1, inplace=True)

In [71]:
map_df.head(3)

,county,latitude,longitude,cluster
16494,anderson,36.20,-84.04,green
16173,bedford,35.59,-86.35,blue
16773,benton,36.23,-88.08,green
